In [ ]:
import re
import pandas as pd
import numpy as np

cols_dif = ['Fecha Programada ', 'Difusión ']
df_dif = pd.read_csv(r'C:\Users\Usuario\Documents\b4d\tigo\esv\tigo_esv\data\difusiones.csv', usecols = cols_dif)

df_dif.columns

Index(['Fecha Programada ', 'Difusión '], dtype='object')

In [2]:
# rename columns
df_dif.rename(columns={'Fecha Programada ': 'scheduled_date', 'Difusión ': 'difusion_name'}, inplace=True)
df_dif.columns

Index(['scheduled_date', 'difusion_name'], dtype='object')

In [3]:

def f_classification_difussions(data_str):
    match data_str:
        case "Primer envio de Producci'on":
            return "Primera Base"
        case "Base II - Tigo EV" | "Base II - Tigo EV Envio 2" | "Base II - Tigo EV - Envio 3":
            return "Segunda Base 120824 1200"
        case "BASE III - Envio 1 AM" | "BASE III - Envio 2 AM" | "BASE III - Envio 3 - pm" | "BASE III - Envio 4 AM":
            return "Tercera base"
        case "Enivo III - Producción" | "Envio III - Producción AM" | "Envio III - Envio AM 2" | "Prueba IV":
            return "Fijo Personas Natural"
        case "Envio 6 | Natural Fijo":
            return "Natural Juridico 141024"
        case "difusiones prueba cambio plan natural movil" | "RE: difusion_tigoev_natural_movil_prueba_281124" | "difusion pruebas tigo ev redireccion":
            return "Cambio Plan Movil 051224"
        case "difusion_1230_lote1" | "difusion_movil_1230_lote2" | "difusion_movil_1230_lote3" | "difusion_movil_1230_lote4" | "difusion_movil_1230_lote5":
            return "Base 210125 1230"
        case "difusion_grupo_1700_lote1" | "difusion_1700_movil_lote2" | "difusion_1700_movil_lote3" | "difusion_1700_movil_lote4" | "difusion_lote_1700_lote5":
            return "Base 210125 1700"
        case _:
            return 'Pruebas'


df_dif['difusion_name_base'] = df_dif['difusion_name'].apply(lambda x: f_classification_difussions(x))
df_dif.columns

Index(['scheduled_date', 'difusion_name', 'difusion_name_base'], dtype='object')

In [4]:
# 
df_dif['scheduled_date'] = df_dif.groupby(['difusion_name_base'])['scheduled_date'].transform('first')
df_dif.shape

(1817, 3)

In [5]:
df_dif = df_dif.drop_duplicates()
df_dif.shape

(40, 3)

### Difusiones unicas

In [6]:
# drop difusion name column  
df_dif_unq = df_dif.drop(columns=['difusion_name'])
df_dif_unq.columns

Index(['scheduled_date', 'difusion_name_base'], dtype='object')

In [7]:
# drop duplicates
df_dif_unq.drop_duplicates(inplace=True)
df_dif_unq.shape

(9, 2)

In [8]:
# change type data scheduled_date
df_dif_unq['scheduled_date'] = pd.to_datetime(df_dif_unq['scheduled_date'])
df_dif_unq['date'] = df_dif_unq['scheduled_date'].dt.date
df_dif_unq['hour'] = df_dif_unq['scheduled_date'].dt.hour
df_dif_unq['minute'] = df_dif_unq['scheduled_date'].dt.minute

In [9]:
# relocation difusion
df_difusion_unq = df_dif_unq[['difusion_name_base', 'scheduled_date', 'date', 'hour', 'minute']]
df_difusion_unq.shape

(9, 5)

In [10]:
df_difusion_unq.head(3)

,difusion_name_base,scheduled_date,date,hour,minute
0,Pruebas,2024-07-04 13:33:00,2024-07-04,13,33
16,Primera Base,2024-07-15 08:28:01,2024-07-15,8,28
101,Segunda Base 120824 1200,2024-08-12 10:00:00,2024-08-12,10,0


In [ ]:
# export to csv to load table

# Associated difusion with caso id

In [11]:
# difusiones con casos id
cols_dif_caso = ['Difusión ', 'Caso ID ', 'Código de Error ', 'Mensaje de Error ', 'Parámetros Extra ']
df_dif_caso = pd.read_csv(r'C:\Users\Usuario\Documents\b4d\tigo\esv\tigo_esv\data\difusiones.csv', usecols = cols_dif_caso)

In [12]:
# rename columns
df_dif_caso.rename(columns={'Difusión ': 'difusion_name', 'Caso ID ': 'caso_id', 'Código de Error ': 'error_code', 'Mensaje de Error ': 'error_msg', 'Parámetros Extra ': 'params_extra'}, inplace=True)
df_dif_caso.columns

Index(['difusion_name', 'caso_id', 'error_code', 'error_msg', 'params_extra'], dtype='object')

In [13]:
# merge difusiones unicos y casos, para crear difusion base
df_difusion_base = df_dif_caso.merge(df_dif, on='difusion_name', how='inner')
df_difusion_base.columns

Index(['difusion_name', 'caso_id', 'error_code', 'error_msg', 'params_extra',
       'scheduled_date', 'difusion_name_base'],
      dtype='object')

In [14]:
df_difusion_base = df_difusion_base[['difusion_name', 'difusion_name_base', 'caso_id', 'scheduled_date','error_code', 'error_msg', 'params_extra']]
df_difusion_base.columns

Index(['difusion_name', 'difusion_name_base', 'caso_id', 'scheduled_date',
       'error_code', 'error_msg', 'params_extra'],
      dtype='object')

In [15]:
df_difusion_base.shape

(1817, 7)

In [16]:
df_difusion_base.head(3)

,difusion_name,difusion_name_base,caso_id,scheduled_date,error_code,error_msg,params_extra
0,Nicolas Martinez - B4D,Pruebas,7008182,2024-07-04 13:33:00,0,NaN,"{""first_name"":""Nicolas"",""last_name"":""Martinez""..."
1,Prueba Melvin,Pruebas,7008183,2024-07-04 13:33:00,0,NaN,"{""first_name"":""Melvin"",""last_name"":""Velasquez""..."
2,RE: Prueba Melvin,Pruebas,7008184,2024-07-04 13:33:00,0,NaN,"{""first_name"":""Melvin"",""last_name"":""Velasquez""..."


# Associated with messages

In [17]:
# Initialize
#cols_msg = ['Fecha ','Tipo Mensaje', 'Asunto ','Grupo ', 'Contenido ', 'Caso ID ']

df_msg = pd.read_csv(r'C:\Users\Usuario\Documents\b4d\tigo\esv\tigo_esv\data\message.csv')
df_msg.shape

(4460, 18)

In [18]:
df_msg.columns

Index(['Fecha ', 'Canal ', 'Tipo Mensaje', 'Tipo Contenido', 'Asunto ',
       'Contenido ', 'Desde Cuenta ', 'Desde Contacto ', 'A Cuenta ',
       'A Contacto ', 'Caso ID ', 'Campaña ', 'Grupo ', 'Usuario ', 'Agente ',
       'RT ', 'Externo ID ', 'Externo ID (origen) '],
      dtype='object')

In [19]:
# rename columns
df_msg.rename(columns={'Fecha ': 'msg_datetime', 'Tipo Mensaje': 'msg_tp', 'Asunto ': 'msg_subject ', 'Contenido ': 'msg_content', 'Caso ID ': 'caso_id', 'Grupo ': 'msg_group', 'Desde Cuenta ': 'from_acc',
                       'A Cuenta ': 'to_acc'}, inplace=True)
df_msg.columns

Index(['msg_datetime', 'Canal ', 'msg_tp', 'Tipo Contenido', 'msg_subject ',
       'msg_content', 'from_acc', 'Desde Contacto ', 'to_acc', 'A Contacto ',
       'caso_id', 'Campaña ', 'msg_group', 'Usuario ', 'Agente ', 'RT ',
       'Externo ID ', 'Externo ID (origen) '],
      dtype='object')

In [20]:
# only difusion message
df_msg = df_msg.merge(df_difusion_base, how='left', on='caso_id')[['difusion_name_base', 'msg_datetime', 'msg_tp', 'msg_subject ', 'msg_content', 'caso_id', 'msg_group', 'error_msg' ,'from_acc', 'to_acc']]
df_msg.shape

(4460, 10)

In [21]:
df_msg.columns

Index(['difusion_name_base', 'msg_datetime', 'msg_tp', 'msg_subject ',
       'msg_content', 'caso_id', 'msg_group', 'error_msg', 'from_acc',
       'to_acc'],
      dtype='object')

In [22]:
# change datatype
df_msg['msg_datetime'] = pd.to_datetime(df_msg['msg_datetime'])

df_msg['msg_date'] = df_msg['msg_datetime'].dt.date

# filtramos solo difusiones
# Ya no filtramos por que si un usuario responde despues del tiempo configurado, se cuenta como un nuevo caso
#df_msg = df_msg[df_msg['msg_group']=='Bot Anfitrion']

df_msg.shape

(4460, 11)

In [23]:

# filtramos solo salientes
#df_msg = df_msg[df_msg['msg_tp']=='Saliente']

# Quitando la columna contenidos null
df_msg = df_msg[df_msg['msg_content'].notna()]

# change datatype 
df_msg['error_msg'] = df_msg['error_msg'].astype(str)

# change datatype
df_msg['msg_content'] = df_msg['msg_content'].astype(str)

df_msg.shape

(4451, 11)

In [24]:

# poblar new column to get customer number
df_msg['cstmr_nmbr'] = np.where(df_msg['msg_tp'] == 'Entrante', df_msg['from_acc'], df_msg['to_acc'])
df_msg.shape

(4451, 12)

In [25]:
df_msg.tail(3)

,difusion_name_base,msg_datetime,msg_tp,msg_subject,msg_content,caso_id,msg_group,error_msg,from_acc,to_acc,msg_date,cstmr_nmbr
4455,NaN,2025-02-10 12:26:04,Entrante,Hola buen día,Hola buen día,7010045,Inbox Tigo SV,nan,50360178359,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-02-10,50360178359
4456,NaN,2025-02-10 12:26:27,Entrante,Necesito ayuda con mi linea,Necesito ayuda con mi linea,7010045,Inbox Tigo SV,nan,50360178359,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-02-10,50360178359
4459,NaN,2025-02-17 13:27:54,Entrante,Buen día,Buen día,7010046,Inbox Tigo SV,nan,50372409987,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-02-17,50372409987


In [26]:
df_msg.columns

Index(['difusion_name_base', 'msg_datetime', 'msg_tp', 'msg_subject ',
       'msg_content', 'caso_id', 'msg_group', 'error_msg', 'from_acc',
       'to_acc', 'msg_date', 'cstmr_nmbr'],
      dtype='object')

In [27]:
# Quitando la columna contenidos null
df_msg[df_msg['difusion_name_base']==""] = np.nan

In [ ]:
# quitando las que son null
#df_msg_1 = df_msg[df_msg['count'].notna()]

In [28]:
# Ordenamos por numero customer, campaign name base y fecha
df_msg.sort_values(by=['cstmr_nmbr', 'difusion_name_base', 'msg_datetime'], inplace=True)
df_msg.head(3)

,difusion_name_base,msg_datetime,msg_tp,msg_subject,msg_content,caso_id,msg_group,error_msg,from_acc,to_acc,msg_date,cstmr_nmbr
4454,NaN,2025-02-10 04:04:20,Entrante,Hola,Hola,7010044,Inbox Tigo SV,nan,34623156993,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-02-10,34623156993
3270,Base 210125 1700,2025-01-21 17:00:31,Saliente,¡hola NAPOLEON ANTONIO BIGIT AHUES :tada:! En...,¡hola NAPOLEON ANTONIO BIGIT AHUES :tada:!\nE...,7009700,Bot Anfitrion,nan,f3fffb210f29649ae4ae76abe92d90fff3545f40,50360000532,2025-01-21,50360000532
4446,NaN,2025-01-28 16:08:54,Entrante,No estoy interesado,No estoy interesado,7010038,Inbox Tigo SV,nan,50360000532,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-01-28,50360000532


In [29]:
# fill column 'difusion_name_base' para usuarios que iniciarion como nuevo caso
df_msg['difusion_name_base'] = df_msg['difusion_name_base'].ffill()

In [30]:
# map errores
map_error = {
    r'Undeliverable|undeliverable' : 'No entregable',
    r'experiment'                  : 'Experimento'
}


def f_mapeo_error(response):
    for pattern, nodo in map_error.items():
        if re.search(pattern, response, re.IGNORECASE):
            return nodo
    return '-1'

df_msg['error_tp'] = df_msg['error_msg'].apply(lambda x: f_mapeo_error(x))

df_msg.dtypes

difusion_name_base            object
msg_datetime          datetime64[ns]
msg_tp                        object
msg_subject                   object
msg_content                   object
caso_id                        int64
msg_group                     object
error_msg                     object
from_acc                      object
to_acc                        object
msg_date                      object
cstmr_nmbr                    object
error_tp                      object
dtype: object

In [31]:
# Mapeo nodos
nodos_res = {
    r'^\¡En Tigo Business impulsamos|^\¡hola'   : '1', # Esta interesado
    r'^Aumenta los beneficios|\¡Excelente'      : '2',
    r'^Elige el plan que sea|:one:*Op'          : '2',
    r'^Yo|^recibe'                              : '3', # Primera condicion
    r'Favor ingresar su numero|^:gift:'         : '4', # Segunda condicion  
    r'^\¡GRACIAS POR CONFIAR EN NOSOTROS|^\¡Gracias|en las próximas':'5',
    r'^Servicio al Cliente Tigo Business|^Todo listo|en las próximas':'6',
    r'^\¿Estas seguro'       : '51',
    r'^Recuerda que'        : '52',
    r'^Aún estas'           : '53',
    r'^Por favor, continua con el proceso': '61',
    r'^\*\¡Estamos para'       : '71',    # Ayuda
    r'^DUI ingresado incorrecto': '72', # DUI inconrrecto
}

def f_mapeo_nodo(response, map_data):
    for pattern, nodo in map_data.items():
        if re.search(pattern, response, re.IGNORECASE):
            return nodo
    return '-1'

df_msg['nodo'] = df_msg['msg_content'].apply(lambda x: f_mapeo_nodo(x, nodos_res))

df_msg.dtypes

difusion_name_base            object
msg_datetime          datetime64[ns]
msg_tp                        object
msg_subject                   object
msg_content                   object
caso_id                        int64
msg_group                     object
error_msg                     object
from_acc                      object
to_acc                        object
msg_date                      object
cstmr_nmbr                    object
error_tp                      object
nodo                          object
dtype: object

In [ ]:
df_msg_nodo = df_msg.groupby('nodo', as_index=False).size()

In [33]:
df_msg.head(3)

,difusion_name_base,msg_datetime,msg_tp,msg_subject,msg_content,caso_id,msg_group,error_msg,from_acc,to_acc,msg_date,cstmr_nmbr,error_tp,nodo
4454,NaN,2025-02-10 04:04:20,Entrante,Hola,Hola,7010044,Inbox Tigo SV,nan,34623156993,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-02-10,34623156993,-1,-1
3270,Base 210125 1700,2025-01-21 17:00:31,Saliente,¡hola NAPOLEON ANTONIO BIGIT AHUES :tada:! En...,¡hola NAPOLEON ANTONIO BIGIT AHUES :tada:!\nE...,7009700,Bot Anfitrion,nan,f3fffb210f29649ae4ae76abe92d90fff3545f40,50360000532,2025-01-21,50360000532,-1,1
4446,Base 210125 1700,2025-01-28 16:08:54,Entrante,No estoy interesado,No estoy interesado,7010038,Inbox Tigo SV,nan,50360000532,f3fffb210f29649ae4ae76abe92d90fff3545f40,2025-01-28,50360000532,-1,-1


In [34]:
df_msg_bot = df_msg[df_msg['msg_tp']=='Saliente']
df_msg_bot.shape

(3478, 14)

In [35]:
df_msg_bot['msg_date_init'] = df_msg_bot.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first') 
df_msg_bot['msg_date_last'] = df_msg_bot.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('last') 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\2630239149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_bot['msg_date_init'] = df_msg_bot.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\2630239149.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_bot['msg_date_last'] = df_msg_bot.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('last')


In [36]:
# Set column nodo flow
df_msg_bot['caso_id'] = df_msg_bot['caso_id'].astype(str)

df_msg_tmp = df_msg_bot.groupby(['cstmr_nmbr', 'difusion_name_base','msg_date_init', 'msg_date_last','error_tp'], as_index=False).agg({'nodo':lambda x: list(x), 'caso_id': lambda x: '_'.join(set(x))})
#df_msg_tmp.rename(columns={'nodo': 'nodo_flow'}, inplace=True)

#df_msg_tmp.head(3)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\3117895906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_bot['caso_id'] = df_msg_bot['caso_id'].astype(str)


In [37]:
df_msg_tmp.shape

(1800, 7)

In [38]:
df_msg_user = df_msg[df_msg['msg_tp']=='Entrante']
df_msg_user.shape

(973, 14)

In [40]:
# Mapeo respuestas
nodos_response = {
    r'Si, estoy interesado'   : '1',
    r'No estoy Interesado|No estoy interesado'    : '2',
    r'monto|iva|costo|barato|precio' : '3',
    r'nombre|mal nombre|papa' : '4',
    r'necesito| solo necesito': '5',
    r'no necesito': '6',
    r'\d+': '7',
    r'Opción 1' : '21',
    r'Opción 2' : '22',
    r'Opción 3' : '23',
}

def f_mapeo_nodo_1(response, map_data):
    for pattern, nodo in map_data.items():
        if re.search(pattern, response, re.IGNORECASE):
            return nodo
    return '-1'

df_msg_user['answer'] = df_msg_user['msg_content'].apply(lambda x: f_mapeo_nodo_1(x, nodos_response))

df_msg_user.shape

df_msg['answer'] = df_msg['msg_content'].apply(lambda x: f_mapeo_nodo_1(x, nodos_response))


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\657035730.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_user['answer'] = df_msg_user['msg_content'].apply(lambda x: f_mapeo_nodo_1(x, nodos_response))


In [41]:
df_msg_user['msg_date_init'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first') 
df_msg_user['msg_date_last'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('last') 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\302551598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_user['msg_date_init'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('first')
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\302551598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_user['msg_date_last'] = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base'], as_index=False)['msg_datetime'].transform('last')


In [42]:
# 
df_msg_user['caso_id'] = df_msg_user['caso_id'].astype(str)

df_msg_tmp_1 = df_msg_user.groupby(['cstmr_nmbr', 'difusion_name_base','msg_date_init', 'msg_date_last', 'error_tp'], as_index=False).agg({'answer':lambda x: list(x), 'caso_id': lambda x: '_'.join(set(x))})
df_msg_tmp_1.rename(columns={'answer': 'answer_flow'}, inplace=True)

df_msg_tmp_1.head(3)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16360\3935126414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_msg_user['caso_id'] = df_msg_user['caso_id'].astype(str)


,cstmr_nmbr,difusion_name_base,msg_date_init,msg_date_last,error_tp,answer_flow,caso_id
0,50360000532,Base 210125 1700,2025-01-28 16:08:54,2025-01-28 16:08:54,-1,[2],7010038
1,50360005350,Segunda Base 120824 1200,2024-08-13 10:00:27,2024-08-13 10:00:28,-1,"[-1, -1]",7008399
2,50360023149,Segunda Base 120824 1200,2024-08-14 18:51:44,2024-08-14 18:51:44,-1,[-1],7008513


In [43]:
# Inner join 
#df_msg = df_msg.merge(df_msg_tmp, on=['cstmr_nmbr', 'difusion_name_base'], how='left')
#df_msg.shape
df_msg_tmp_1.columns = ['cstmr_nmbr', 'difusion_name_base', 'user_msg_date_init', 'user_msg_date_last', 'user_error_tp', 'answer_flow', 'caso_id']
df_msg_tmp_1.columns

Index(['cstmr_nmbr', 'difusion_name_base', 'user_msg_date_init',
       'user_msg_date_last', 'user_error_tp', 'answer_flow', 'caso_id'],
      dtype='object')

In [44]:
df_msg_fct = df_msg_tmp.merge(df_msg_tmp_1[['cstmr_nmbr', 'difusion_name_base', 'user_msg_date_init', 'user_msg_date_last',
    'answer_flow', 'caso_id']], on=['cstmr_nmbr', 'difusion_name_base', 'caso_id'], how='left')
df_msg_fct.shape

(1800, 10)

In [45]:
df_msg_fct.head(3)

,cstmr_nmbr,difusion_name_base,msg_date_init,msg_date_last,error_tp,nodo,caso_id,user_msg_date_init,user_msg_date_last,answer_flow
0,50360000532,Base 210125 1700,2025-01-21 17:00:31,2025-01-21 17:00:31,-1,[1],7009700,NaT,NaT,NaN
1,50360001100,Base 210125 1230,2025-01-21 12:39:17,2025-01-21 12:39:17,-1,[1],7009273,NaT,NaT,NaN
2,50360001165,Segunda Base 120824 1200,2024-08-13 10:00:31,2024-08-13 10:00:31,-1,[1],7008434,NaT,NaT,NaN


In [ ]:
df_msg_fct.columns

In [46]:
df_msg_fct.columns = ['cstmr_nmbr', 'difusion_name_base', 'msg_date_init', 'msg_date_last',
       'error_tp', 'nodo_flow', 'caso_id', 'user_msg_date_init',
       'user_msg_date_last', 'answer_flow']

In [47]:
def f_scene(data):
    if '5' in data:
        return 'Conversion'
    elif '5' not in data and len(data) > 1:
        return 'No termino el proceso'
    return 'No inicio el proceso'


df_msg_fct['scene'] = df_msg_fct['nodo_flow'].map(lambda x: f_scene(x))

df_msg_fct.head(3)

,cstmr_nmbr,difusion_name_base,msg_date_init,msg_date_last,error_tp,nodo_flow,caso_id,user_msg_date_init,user_msg_date_last,answer_flow,scene
0,50360000532,Base 210125 1700,2025-01-21 17:00:31,2025-01-21 17:00:31,-1,[1],7009700,NaT,NaT,NaN,No inicio el proceso
1,50360001100,Base 210125 1230,2025-01-21 12:39:17,2025-01-21 12:39:17,-1,[1],7009273,NaT,NaT,NaN,No inicio el proceso
2,50360001165,Segunda Base 120824 1200,2024-08-13 10:00:31,2024-08-13 10:00:31,-1,[1],7008434,NaT,NaT,NaN,No inicio el proceso


In [48]:
def f_last_nodo(data):
    if '5' in data and len(data) > 1:
        return '5 - Conversion'
    elif '4' in data and len(data) > 1:
        return '4 - Problema con DUI'
    elif '3' in data and len(data) > 1:
        return '3 - No confirma'
    elif '2' in data and len(data) > 1:
        return '2 - No elige opcion'
    elif '1' in data and len(data) < 2:
        return '1 - No inicio'
    return '8 - No termino'

df_msg_fct['last_nodo'] = df_msg_fct['nodo_flow'].map(lambda x: f_last_nodo(x))
df_msg_fct.head(3)

,cstmr_nmbr,difusion_name_base,msg_date_init,msg_date_last,error_tp,nodo_flow,caso_id,user_msg_date_init,user_msg_date_last,answer_flow,scene,last_nodo
0,50360000532,Base 210125 1700,2025-01-21 17:00:31,2025-01-21 17:00:31,-1,[1],7009700,NaT,NaT,NaN,No inicio el proceso,1 - No inicio
1,50360001100,Base 210125 1230,2025-01-21 12:39:17,2025-01-21 12:39:17,-1,[1],7009273,NaT,NaT,NaN,No inicio el proceso,1 - No inicio
2,50360001165,Segunda Base 120824 1200,2024-08-13 10:00:31,2024-08-13 10:00:31,-1,[1],7008434,NaT,NaT,NaN,No inicio el proceso,1 - No inicio


In [ ]:
df_temp12 = df_msg_fct.groupby('last_nodo').size()

In [49]:
# Interesado column
df_msg_fct['interest'] = df_msg_fct['answer_flow'].apply(lambda x: '1' in x if isinstance(x, list) else False) 

# Clientes que pudieron pasar del nodo 2 a nodo 3
df_msg_fct['nodo_1_to_2'] = df_msg_fct['nodo_flow'].apply(lambda x: '1' and '2' in x if isinstance(x, list) else False) 

# Clientes que pudieron pasar del nodo 3 a nodo 4
df_msg_fct['nodo_2_to_3'] = df_msg_fct['nodo_flow'].apply(lambda x: '2' and '3' in x if isinstance(x, list) else False) 

# Clientes que pudieron pasar del nodo 4 a nodo 5
df_msg_fct['nodo_3_to_4'] = df_msg_fct['nodo_flow'].apply(lambda x: '3' and '4' in x if isinstance(x, list) else False) 

# Clientes que pudieron pasar del nodo 5 a nodo 6
df_msg_fct['nodo_4_to_5'] = df_msg_fct['nodo_flow'].apply(lambda x: '4' and '5' in x if isinstance(x, list) else False) 

# Clientes que pudieron pasar del nodo 5 a nodo 6
df_msg_fct['nodo_5_to_6'] = df_msg_fct['nodo_flow'].apply(lambda x: '5' and '6' in x if isinstance(x, list) else False) 

In [50]:
df_msg_fct['msg_date_answer_days'] = (df_msg_fct['user_msg_date_init'] - df_msg_fct['msg_date_init']).dt.total_seconds() / (24 * 3600)

In [51]:
def clasificar_tiempo(diferencia_dias):
    if diferencia_dias < 0:
        return 'Respuesta antes del envío'
    elif diferencia_dias == 0:
        return 'Mismo día'
    elif diferencia_dias < 1:
        return '< 1 día'
    elif diferencia_dias < 2:
        return '< 2 días'
    elif diferencia_dias < 3:
        return '< 3 días'
    else:
        return '> 3 días o más'

In [52]:
# Tiempo muerto
df_msg_fct['tiempo_muerto'] = df_msg_fct['msg_date_answer_days'].apply(clasificar_tiempo)

In [53]:
# Ayuda column
df_msg_fct['ayuda'] = df_msg_fct['nodo_flow'].apply(lambda x: '71' in x if isinstance(x, list) else False) 

In [ ]:
# reorder columns 
df_dif = df_dif[['difusion_name', 'difusion_name_base', 'scheduled_date']]
df_dif['scheduled_date'] = pd.to_datetime(df_dif['scheduled_date'])
df_dif.columns
df_dif.to_csv('data/tigo_esv_dif_dim.csv')

In [ ]:
df_difusion_unq['date'] = pd.to_datetime(df_difusion_unq['date'])
df_difusion_unq.to_csv('data/tigo_esv_dif_grp_dim.csv')

In [ ]:
df_difusion_base.columns
# reorder
df_difusion_base = df_difusion_base[['caso_id', 'difusion_name', 'difusion_name_base', 'scheduled_date', 'error_code', 'error_msg', 'params_extra']]

df_difusion_base['scheduled_date'] = pd.to_datetime(df_difusion_base['scheduled_date'])

df_difusion_base.to_csv('data/tigo_esv_caso_dim.csv')

In [54]:
df_msg.columns

df_msg.rename(columns={'msg_subject ': 'msg_subject'}, inplace=True)
# reorder
df_msg = df_msg[['caso_id', 'difusion_name_base', 'msg_datetime', 'msg_date', 'msg_tp', 'msg_subject', 'msg_content', 'msg_group', 'error_msg', 'cstmr_nmbr', 'error_tp', 'nodo', 'answer']]
df_msg['msg_date'] = pd.to_datetime(df_msg['msg_date'])
df_msg.to_csv('data/tigo_esv_caso_msg_dtl.csv')

In [ ]:
df_msg.shape

In [56]:
df_msg_fct.dtypes
df_msg_fct.to_csv('data/tigo_esv_caso_msg_fct.csv')
